<a href="https://colab.research.google.com/github/PietroCaforio/research-biocv-proj/blob/dev/unimodal_ct_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train unimodal CT

In [1]:
!git clone https://github.com/PietroCaforio/research-biocv-proj
!cd research-biocv-proj && git switch dev

Cloning into 'research-biocv-proj'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 138 (delta 66), reused 92 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (138/138), 3.35 MiB | 22.13 MiB/s, done.
Resolving deltas: 100% (66/66), done.
Branch 'dev' set up to track remote branch 'dev' from 'origin'.
Switched to a new branch 'dev'


In [2]:
!cd research-biocv-proj && git pull

Already up to date.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip -o /content/drive/MyDrive/processed57PatientsCPTAC-PDA.zip -d research-biocv-proj/data/

Archive:  /content/drive/MyDrive/processed57PatientsCPTAC-PDA.zip
   creating: research-biocv-proj/data/processed/CT/
   creating: research-biocv-proj/data/processed/CT/C3L-00189/
  inflating: research-biocv-proj/data/processed/CT/C3L-00189/0.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-00189/1.npy  
   creating: research-biocv-proj/data/processed/CT/C3L-00599/
  inflating: research-biocv-proj/data/processed/CT/C3L-00599/0.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-00599/1.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-00599/2.npy  
   creating: research-biocv-proj/data/processed/CT/C3L-00622/
  inflating: research-biocv-proj/data/processed/CT/C3L-00622/0.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-00622/1.npy  
  inflating: research-biocv-proj/data/processed/CT/C3L-00622/2.npy  
   creating: research-biocv-proj/data/processed/CT/C3L-00625/
  inflating: research-biocv-proj/data/processed/CT/C3L-00625/0.npy  
  inflating: 

In [5]:
import sys
from pathlib import Path

# Add the 'data' directory to sys.path
sys.path.append(str(Path('research-biocv-proj').resolve()))
from data.unimodal import *
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import DataLoader

In [6]:
# Instantiate the dataset
dataset = UnimodalCTDataset(split='train', dataset_path = "research-biocv-proj/data/processed/")

# Check the length of the dataset
print(f"Dataset length: {len(dataset)}")

# Check the first few items in the dataset
for i in range(3):
    item = dataset[i]
    print(f"Item {i}:")
    print(f"  Patient ID: {item['patient_id']}")
    print(f"  Frame shape: {item['frame'].shape}")
    print(f"  Label: {item['label']}")

# Check if DataLoader works with the dataset
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Get a batch of data
batch = next(iter(dataloader))

# Check the batch
print(f"Batch patient IDs: {batch['patient_id']}")
print(f"Batch frame shape: {batch['frame'].shape}")
print(f"Batch labels: {batch['label']}")

# Move batch to device (e.g., GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset.move_batch_to_device(batch, device)
print(f"Batch moved to device: {device}")

Dataset length: 2329
Item 0:
  Patient ID: C3L-03632
  Frame shape: (3, 224, 224)
  Label: 0
Item 1:
  Patient ID: C3L-03632
  Frame shape: (3, 224, 224)
  Label: 0
Item 2:
  Patient ID: C3L-03632
  Frame shape: (3, 224, 224)
  Label: 0
Batch patient IDs: ['C3N-01715', 'C3L-03632', 'C3N-01166', 'C3N-03430']
Batch frame shape: torch.Size([4, 3, 224, 224])
Batch labels: tensor([2, 0, 1, 1])
Batch moved to device: cuda


### Train ResNet model

In [29]:
def train(model,lr=0.0001,weight_decay = 0.001, reduce_lr_factor = 0.1):
  optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
  criterion = nn.CrossEntropyLoss()
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor = reduce_lr_factor)
  # Training loop
  num_epochs = 100
  for epoch in range(num_epochs):
      model.train()
      running_loss = 0.0

      for batch in train_loader:
          frames = batch['frame'].float().to(device)
          labels = batch['label'].long().to(device)

          optimizer.zero_grad()
          outputs = model(frames)
          loss = criterion(outputs.logits, labels)

          loss.backward()
          optimizer.step()

          running_loss += loss.item()

      print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

      # Validation loop
      model.eval()
      val_loss = 0.0
      correct = 0
      total = 0
      # Initialize counters for each class (G1, G2, G3)
      correct_per_class = [0, 0, 0]  # For G1, G2, G3
      total_per_class = [0, 0, 0]  # For G1, G2, G3

      with torch.no_grad():
          for batch in val_loader:
              frames = batch['frame'].float().to(device)
              labels = batch['label'].long().to(device)

              outputs = model(frames)
              loss = criterion(outputs.logits, labels)

              val_loss += loss.item()
              _, predicted = torch.max(outputs.logits, 1)
              total += labels.size(0)
              correct += (predicted == labels).sum().item()

              # Calculate accuracy per class
              for i in range(3):  # We have 3 classes: G1 (0), G2 (1), G3 (2)
                  correct_per_class[i] += ((predicted == i) & (labels == i)).sum().item()
                  total_per_class[i] += (labels == i).sum().item()
      scheduler.step(val_loss)
      # Compute total accuracy and per-class accuracy
      total_accuracy = 100 * correct / total
      class_accuracy = [(100 * correct_per_class[i] / total_per_class[i]) if total_per_class[i] > 0 else 0 for i in range(3)]
      print(f"Validation Loss: {val_loss/len(val_loader)}, Total Accuracy: {total_accuracy:.2f}%")
      print(f"Accuracy per class - G1: {class_accuracy[0]:.2f}%, G2: {class_accuracy[1]:.2f}%, G3: {class_accuracy[2]:.2f}%")

In [16]:
import torch.nn as nn
import torch.optim as optim
from transformers import ResNetForImageClassification

In [17]:
train_dataset = UnimodalCTDataset(split='train',dataset_path = "research-biocv-proj/data/processed/" )
val_dataset = UnimodalCTDataset(split='val',dataset_path = "research-biocv-proj/data/processed/")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

### Resnet-50

In [25]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-50')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

In [20]:
train(model)

Epoch 1, Loss: 0.8615876787329373
Validation Loss: 32.5615843385458, Total Accuracy: 55.70%
Accuracy per class - G1: 0.00%, G2: 100.00%, G3: 0.00%
Epoch 2, Loss: 0.5460653398951439
Validation Loss: 45.078371407091616, Total Accuracy: 55.03%
Accuracy per class - G1: 0.00%, G2: 98.80%, G3: 0.00%
Epoch 3, Loss: 0.3530280530860979
Validation Loss: 70.46642629057169, Total Accuracy: 56.04%
Accuracy per class - G1: 0.00%, G2: 90.36%, G3: 12.88%
Epoch 4, Loss: 0.2066208182336533
Validation Loss: 78.61137111485004, Total Accuracy: 77.52%
Accuracy per class - G1: 0.00%, G2: 83.13%, G3: 70.45%
Epoch 5, Loss: 0.12462664666037036
Validation Loss: 90.11593417152763, Total Accuracy: 76.51%
Accuracy per class - G1: 0.00%, G2: 78.92%, G3: 73.48%
Epoch 6, Loss: 0.0865426078591853
Validation Loss: 105.47768404632806, Total Accuracy: 77.18%
Accuracy per class - G1: 0.00%, G2: 79.52%, G3: 74.24%
Epoch 7, Loss: 0.0651686066914707
Validation Loss: 129.5552030812949, Total Accuracy: 75.17%
Accuracy per class

KeyboardInterrupt: 

### Resnet-18

In [26]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-18')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBasicLayer(
              (shortcut): Identity()
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                  (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                  (activation): ReLU()
           

In [30]:
train(model, weight_decay = 0.0)

Epoch 1, Loss: 0.1020891621536639
Validation Loss: 1.4723201919347049, Total Accuracy: 52.01%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 22.73%
Epoch 2, Loss: 0.002461756228737227
Validation Loss: 1.3004954083822668, Total Accuracy: 57.38%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 34.85%
Epoch 3, Loss: 0.0012243360405379575
Validation Loss: 1.451321676396765, Total Accuracy: 56.71%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 33.33%
Epoch 4, Loss: 0.0011813912978427714
Validation Loss: 1.6715782206505536, Total Accuracy: 64.09%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 50.00%
Epoch 5, Loss: 0.0005825575395672237
Validation Loss: 1.5267584381159396, Total Accuracy: 64.43%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 50.76%
Epoch 6, Loss: 0.0008343972872913981
Validation Loss: 1.5266028969548642, Total Accuracy: 66.78%
Accuracy per class - G1: 0.00%, G2: 75.30%, G3: 56.06%
Epoch 7, Loss: 0.00046355306762525784
Validation Loss: 1.6231416193186305, Total Accuracy: 56

KeyboardInterrupt: 

### Resnet-34

In [ ]:
model = ResNetForImageClassification.from_pretrained('microsoft/resnet-34')
model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, UnimodalCTDataset.num_classes) #Adjusting the final layer to the unimodal number of classes

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
train(model)